In [1]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime

# for loading algoseek data
import algoseek_connector
import algoseek_connector.functions as fn

# for algoseek user credentials
from dotenv import load_dotenv
import os

# to manage paths
import os.path
import sys

### Load Algoseek Data

#### Data Fetch

In [3]:
load_dotenv() # load environment variables

jhost = os.getenv('host_julian')
juser = os.getenv('user_julian')
jpassword = os.getenv('password_julian')

ahost = os.getenv('host_angel')
auser = os.getenv('user_angel')
apassword = os.getenv('password_angel')

In [4]:
# Instantiate the session
session = algoseek_connector.Session(ahost, auser, apassword)
session.ping()

resource = algoseek_connector.DataResource(session)

taq_min = resource.datagroups.USEquityMarketData.datasets.TradeAndQuoteMinuteBar

volatile_tech_tickers = ['AMD']
# volatile_tech_tickers = ['AMD', 'SQ', 'TSLA']


taq1min = taq_min.filter(
       (taq_min.Ticker.isin(volatile_tech_tickers)) &
       # ('2022-07-11' >= taq_min.TradeDate <= '2022-07-12')
       ('2022-07-11' == taq_min.TradeDate)
).fetch()

Failed to connect to 54.209.205.114:9000
Traceback (most recent call last):
  File "C:\Users\angel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\clickhouse_driver\connection.py", line 325, in connect
    return self._init_connection(host, port)
  File "C:\Users\angel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\clickhouse_driver\connection.py", line 289, in _init_connection
    self.socket = self._create_socket(host, port)
  File "C:\Users\angel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\clickhouse_driver\connection.py", line 261, in _create_socket
    raise err
  File "C:\Users\angel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\clickhouse_driver\connection.py", line 252, in _create

ExecutionError: (54.209.205.114:9000)

In [17]:
# save backup file

os.chdir('C:/Users/angel/Documents/Documents/GitHub/AlgoSeek-Notebooks')

taq1min.to_csv('AlgoSeekNotebooks/datasets/taq_1min_volatile_tech_backup.csv')

#### Data Preprocessing

In [13]:
# create backup variable for data prep
taq = taq1min

taq.TradeDate = pd.to_datetime(taq.TradeDate, format="%Y-%m-%d")
taq.set_index(['BarDateTime', 'Ticker'], inplace=True)

taq.sort_index().groupby('BarDatTime', group_keys=False)

taq['date'] = pd.to_datetime(taq1min.index.get_level_values('BarDateTime').date)

KeyError: 'BarDatTime'

In [18]:
taq.head()

,,TradeDate,SecId,OpenBarTimeOffset,OpenBidPrice,OpenBidSize,OpenAskPrice,OpenAskSize,FirstTradeTimeOffset,FirstTradePrice,FirstTradeSize,...,FinraVolumeWeightPrice,UptickVolume,DowntickVolume,RepeatUptickVolume,RepeatDowntickVolume,UnknownTickVolume,TradeToMidVolWeight,TradeToMidVolWeightRelative,TimeWeightBid,TimeWeightAsk
BarDateTime,Ticker,,,,,,,,,,,,,,,,,,,,,
2007-01-03 04:15:00-05:00,AMD,2007-01-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,20.25,20.49
2007-01-03 04:16:00-05:00,AMD,2007-01-03,0,0,20.25,100,20.49,1200,0,0,0,...,0,0,0,0,0,0,0,0,20.25,20.49
2007-01-03 04:17:00-05:00,AMD,2007-01-03,0,0,20.25,100,20.49,1200,0,0,0,...,0,0,0,0,0,0,0,0,20.25,20.49
2007-01-03 04:18:00-05:00,AMD,2007-01-03,0,0,20.25,100,20.49,1200,0,0,0,...,0,0,0,0,0,0,0,0,20.25,20.49
2007-01-03 04:19:00-05:00,AMD,2007-01-03,0,0,20.25,100,20.49,1200,0,0,0,...,0,0,0,0,0,0,0,0,20.25,20.49


In [ ]:
taq.to_csv('AlgoSeekNotebooks/datasets/taq_1min_volatile_tech.csv')

### Load Local CSV

In [ ]:
class customCSV(btfeed.GenericCSVData):
    params = (
        ('dtformat', '%m-%d-%Y'),
        ('tmformat', '%H:%M:%S.%f'),
        ('datetime', 0),
        ('time', 1),
        ('open', 2),
        ('high', 3),
        ('low', 4),
        ('close', 5),
        ('volume', -1),
        ('openinterest', -1),
    )



### Strategy Logic

#### Bollinger Band Mean Reversion

- Buy when standard deviation <= -2
- Sell when standard deviation >= 2
- Liquidate when standard deviation <= 0.5 or 0

In [ ]:
class MeanReversion(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])


'''
def next(self):
    for i, d in enumerate(self.datas):
        if not self.getposition(d).size:
            if self.crossovers[i] > 0: 
                self.buy(data = d)
        elif self.crossovers[i] < 0: 
            self.close(data = d)
'''

In [ ]:
'''

Strategy Pseudocode



'''

### Run Cerebro Engine

In [20]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()

    # strats = cerebro.optstrategy(MeanReversion)

    # data = bt.feeds.YahooFinanceCSVData(
    #     dataname=datapath,
    #     # Do not pass values before this date
    #     fromdate=datetime.datetime(2000, 1, 1),
    #     # Do not pass values before this date
    #     todate=datetime.datetime(2000, 12, 31),
    #     # Do not pass values after this date
    #     reverse=False)

    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'AlgoSeekNotebooks/datasets/taq_1min_volatile_tech.csv')

    # Create a Data Feed
    data = bt.feeds.CSVDataBase(
        dataname=datapath,
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    # cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Run strategy
    # cerebro.run(maxcpus=1)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

AttributeError: module 'backtrader.feeds' has no attribute 'CSVDataBase'

### Benchmark Strategy

- Buy and hold each company with equal weighted portfolio allocation